In [1]:
import ipywidgets.widgets as widgets

In [2]:
controller=widgets.Controller(index=0)

In [3]:
display(controller)

Controller()

In [4]:
from jetracer.nvidia_racecar import NvidiaRacecar

WARNNIG: Jetson.GPIO library has not been verified with this carrier board,


In [5]:
import traitlets

In [6]:
car=NvidiaRacecar()

In [7]:
car.throttle_gain=0.1
car.throttle = 0
car.steering_offset=0
car.steering=0
car.steering_gain=-1



In [8]:
left_link=traitlets.dlink((controller.axes[2],'value'),(car,'steering'), transform=lambda x:x)

In [9]:
right_link=traitlets.dlink((controller.axes[1],'value'),(car,'throttle'), transform=lambda x:-x)

In [10]:
from jetcam.utils import bgr8_to_jpeg
from IPython.display import display
import ipywidgets
from jetcam.csi_camera import CSICamera
camera = CSICamera(width=224, height=224)
camera.running = True

RuntimeError: Could not initialize camera.  Please see error trace.

In [ ]:
widget = ipywidgets.Image(format='jpeg', width=camera.width, height=camera.height)
traitlets.dlink((camera, 'value'), (widget, 'value'), transform=bgr8_to_jpeg)

display(widget)

In [ ]:
image = widgets.Image(format='jpeg', width=100, height=300)

In [ ]:
display(image)

In [ ]:
from jetcam.csi_camera import CSICamera
# from jetcam.usb_camera import USBCamera

camera = CSICamera(width=400, height=400)
# camera = USBCamera(width=224, height=224)

camera.running = True

In [ ]:
import traitlets
import cv2
from PIL import Image
import numpy as np

def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)


In [ ]:
import traitlets
from traitlets.config.configurable import Configurable
from traitlets import Unicode, Dict, List, Instance
import time

class Heartbeat(Configurable):
    period = traitlets.Float(default_value=1.0)
    status = traitlets.Unicode(default_value="alive")

def handle_heartbeat_status(change):
    if change['new'] == "dead":
        camera_link.unlink()
        left_link.unlink()
        right_link.unlink()
        robot.stop()

heartbeat = Heartbeat(period=0.5)

# attach the callback function to heartbeat status
heartbeat.observe(handle_heartbeat_status, names='status')

In [ ]:
camera.unobserve_all()